In [3]:
from pathlib import Path
from higgs.consts import COLNAMES, DATAROOT
import pickle

In [4]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 30

np.set_printoptions(
    edgeitems=30, 
    linewidth=100000, 
    precision=3,
    suppress=True
)

NameError: name 'pd' is not defined

In [ ]:
filename = DATAROOT / "SAMPLE_HIGGS.csv"
higgs = pd.read_csv(filename, header=None, names=COLNAMES)

In [ ]:
higgs.info()

In [ ]:
trainset_size = int(len(higgs) * 0.9)
valset_size = len(higgs) - trainset_size
print(f"trainset size: {trainset_size:,}, valset size: {valset_size:,}")

In [ ]:
trainset_df = higgs[:trainset_size]
valset_df = higgs[trainset_size:]
print(f"trainset size: {len(trainset_df)}, valset size: {len(valset_df)}")

In [ ]:
class PandasDataset(td.Dataset):
    def __init__(self, df):
        super().__init__()
        self._X = df[COLNAMES[1:]].values
        self._y = df["label"].values.astype(int)
        
    def __len__(self):
        return self._y.shape[0]
    
    def __getitem__(self, idx):
        return self._X[idx], self._y[idx]

In [ ]:
trainset = PandasDataset(trainset_df)
valset = PandasDataset(valset_df)

In [ ]:
class HiggsDataModule(pl.LightningDataModule):
    def __init__(self, dataroot=DATAROOT, infile="SAMPLE_HIGGS.csv", trainset_prop=0.9, hp=None):
        super().__init__()
        self.trainset, self.valset = None, None
        self.hparams = hp
        self._dataroot = dataroot
        self._infile = infile
        self._trainset_prop = trainset_prop
        self._trainfile = self._dataroot / "train.pkl"
        self._valfile = self._dataroot / "val.pkl"
        
    def prepare(self):
        if (self._trainfile.exists() and self._valfile.exists()):
            return
            
        higgs = pd.read_csv(self._filename, header=None, names=COLNAMES)
        
        trainset_size = int(len(higgs) * self._trainset_prop)
        train_df = higgs[:trainset_size]
        train_X = train_df[COLNAMES[1:]].values
        train_y = train_df["label"].values.astype(int)
        with open(self.trainfile, "wb") as f:
            pickle.dump(f, {"X": train_X, "y": train_y})
        
        valset_size = len(higgs) - trainset_size
        val_df = higgs[trainset_size:]
        val_X = val_df[COLNAMES[1:]].values
        val_y = val_df["label"].values.astype(int)
        with open(self._valfile, "wb") as f:
            pickle.dump(f, {"X": val_X, "y": val_y})
        
    def setup(self, stage):
        if stage == "fit":
            
        